## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo  
        self.nombre = nombre  
        self.tipo = tipo      
        self.precio = precio  
        self.cantidad = cantidad  

    @property
    def codigo(self):
        return self._codigo

    @codigo.setter
    def codigo(self, valor):
        if len(valor) == 4 and valor.isdigit():
            self._codigo = valor
        else:
            print("El código debe tener exactamente 4 dígitos y ser numérico.")

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if len(valor) <= 100 and len(valor) >= 1 :
            self._nombre = valor
        else:
            print("El nombre tiene demasiados caracteres. ¿Estás seguro que lo pusiste bien?")

    @property
    def tipo(self):
        return self._tipo

    @tipo.setter
    def tipo(self, valor):
        if len(valor) <= 20:
            self._tipo = valor
        else:
            print("Te pasaste del límite de caracteres para el tipo de producto.")

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor >= 10 and valor <= 10000:
            self._precio = valor
        else:
            print("Precio no valido porfavor ingrese el valor correcto")

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if  valor >=0 and valor <= 100:
            self._cantidad = valor
        else:
            print("Se puede colocar un máximo de 100 cantidades por producto y un mínimo de 0.")

    def valorTotal(self):
        return self.precio*self.cantidad

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Precio no valido porfavor ingrese el valor correcto
Se puede colocar un máximo de 100 cantidades por producto y un mínimo de 0.
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto, cantidad):
        return (producto.codigo in self.codigos) or (producto.tipo in self.tipos)

class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="Descuento 10%", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad): 
            return cantidad * producto.precio * self.descuento / 100
        return 0

class Oferta2x1(Oferta):
    def __init__(self, descripcion="Oferta 2x1", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):  
            if cantidad == 1:
                return 0
            elif cantidad % 2 == 0:
                return (cantidad // 2) * producto.precio
            else:
                return (cantidad // 2) * producto.precio
        return 0

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []  
        self.ofertas = [] 
    
    @classmethod
    def leer(cls, archivo: str) -> 'Catalogo':
        catalogo = cls()
        with open(archivo, "r") as file:
            next(file) 
            for linea in file:
                codigo, nombre, precio, tipo, cantidad = linea.strip().split(",")
                producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                catalogo.productos.append(producto)
        return catalogo
    
    def guardar(self, archivo):
        with open(archivo, 'w', newline="") as nuevo_Archivo:
                fieldnames = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
                writer = csv.DictWriter(nuevo_Archivo, fieldnames=fieldnames)

                writer.writeheader()  
                for producto in self.productos:
                    
                    writer.writerow({
                        'codigo': producto.codigo,
                        'nombre': producto.nombre,
                        'precio': producto.precio,
                        'tipo': producto.tipo,
                        'cantidad': producto.cantidad
                    })
    
    def agregar(self, *productos):
        for producto in productos:
            if isinstance(producto, Producto):
                self.productos.append(producto)
            else:
                print(f"Lo que ingresaste de {producto} No se agregara.")


   
    def vender(self, producto, cantidad):
        prod = self.buscar(producto.codigo)
        if prod:
            if prod.cantidad >= cantidad:
                prod.cantidad -= cantidad
            else:
                print(f"No hay {producto.nombre}. Solo quedan {prod.cantidad}.")
        else:
            print(f"El articulo {producto.codigo} No se encuentra disponible.")

    def calcularDescuento(self, producto, cantidad):
        descuento = next((oferta.calcularDescuento(producto, cantidad) 
         for oferta in self.ofertas 
             if oferta.esAplicable(producto, cantidad)), 0)
        return descuento

    
    
    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
         if oferta.esAplicable(producto, cantidad):  
            return oferta
        return None
    def registrarOferta(self,oferta):
        self.ofertas.append(oferta)
    
    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None
   
    @property
    def cantidadProductos(self):
        return len(self.productos)
    
    @property
    def cantidadUnidades(self):
        ct = sum(producto.cantidad for producto in self.productos)
        return ct
    
    @property
    def valorTotal(self):
        vt=sum(producto.precio * producto.cantidad for producto in self.productos)
        return vt
    
    @property
    def precioPromedio(self):
        if self.cantidadProductos == 0:
            return 0
        return self.valorTotal / self.cantidadProductos
    
  
    def informe(self):
        tipos = set(producto.tipo for producto in self.productos)
        tipos_formateados = "\n                  • ".join(tipos)
        informe = [
            f"Cantidad de productos: {self.cantidadProductos}",
            f"Cantidad de unidades: {self.cantidadUnidades}",
            f"Precio Promedio: {self.precioPromedio:.2f}",
            f"Valor total: {self.valorTotal:.2f}",
            f"Tipos de productos: \n                  • {tipos_formateados}"
        ]
        ahorro_descuento = sum(
            self.calcularDescuento(producto, producto.cantidad)
         for producto in self.productos
            if self.buscarOferta(producto, producto.cantidad) and self.buscarOferta(producto, producto.cantidad).descripcion == "Descuento 10%"
        )
        ahorro_2x1 = sum(
        (producto.cantidad - ((producto.cantidad // 2) + (producto.cantidad % 2))) * producto.precio
        for producto in self.productos
        if self.buscarOferta(producto, producto.cantidad) and self.buscarOferta(producto, producto.cantidad).descripcion == "Oferta 2x1"
        )
        if self.ofertas:
            informe.append(f"Ofertas: {ahorro_descuento:.2f}")
            informe.append(f"Oferta 2x1: {ahorro_2x1:.2f}")
        return '\n'.join(informe)

In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente: 
    def __init__(self, nombre, cuit): 
        self.nombre = nombre
        self.cuit=cuit



    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if len(valor) <= 100 and len(valor) >= 1 :
            self._nombre = valor
        else:
            print("Se paso del limite de caracteres para tu nombre")

    @property
    def cuit(self):
        return self._cuit
    
    @cuit.setter
    def cuit (self,valor):
        if len(valor) == 13 and valor[2] == '-' and valor[11] == '-' and valor [:2].isdigit() and valor [:2].isdigit() and valor [3:11].isdigit() and valor [12].isdigit():
            self._cuit = valor 
        elif len (valor) == 11 and valor.isdigit():
            self._cuit = f"{valor[:2]}-{valor[2:10]}-{valor[10]}"
        else:
            print ("pusiste mal el cuit")

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Se paso del limite de caracteres para tu nombre
pusiste mal el cuit
pusiste mal el cuit
Prueba pasada exitosamente!


In [9]:
import datetime

class Factura:
    
    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.datetime.now().strftime("%d/%m/%Y")
        self.cliente = cliente
        self.catalogo = catalogo
        self.items = []  
        self.subtotal = 0
        self.descuentos = 0
    
    ultimafactura = None

    @classmethod
    def ultimaFactura(cls, numero):
        cls.ultima_factura = numero

    @classmethod
    def nuevoNumero(cls):
        cls.ultima_factura += 1
        return cls.ultima_factura

   

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(item[1] for item in self.items)

    @property
    def total(self):
        return self.subtotal - self.descuentos

    
    def agregar(self, producto, cantidad):
        self.catalogo.vender(producto, cantidad)
        existente = next((i for i, (prod, _) in enumerate(self.items) if prod.codigo == producto.codigo), None)
    
        if existente is not None:
            self.items[existente] = (self.items[existente][0], self.items[existente][1] + cantidad)
        else:
            self.items.append((producto, cantidad))

        self.actualizarTotales()

    def actualizarTotales(self):
        self.subtotal = sum( producto.precio * cantidad for producto, cantidad in self.items)
    
        self.descuentos = sum(self.catalogo.calcularDescuento(producto, cantidad) for producto, cantidad in self.items)
        
    def imprimir(self):
        Comprobantecompra=f"Factura n°: {self.numero}\n"
        Comprobantecompra+=f"Fecha: {self.fecha}\n"
        Comprobantecompra+=f"Cliente: {self.cliente.nombre}\n"
        Comprobantecompra+=f"Productos:\n"
        for prod, cant in self.items:
            precioUnitario=prod.precio
            subtotalProd = precioUnitario * cant
            descuentoProd = self.catalogo.calcularDescuento(prod,cant)
            ofertasProd = [oferta.descripcion for oferta in self.catalogo.ofertas if oferta.esAplicable(prod,cant)]
            Comprobantecompra += f"- {cant} {prod.nombre} x ${precioUnitario} = ${subtotalProd:.2f} \n"
            if descuentoProd > 0:
                Comprobantecompra+=f"- Descuento: ${descuentoProd:.2f}"
            if ofertasProd:
                Comprobantecompra+= f"{', '.join(ofertasProd)}\n"
            else:
                Comprobantecompra+="\n"
            

        Comprobantecompra+=f"Subtotal: ${self.subtotal:.2f}\n"
        Comprobantecompra+=f"Descuentos: ${self.descuentos:.2f}\n"
        Comprobantecompra+=f"TOTAL: ${self.total:.2f}\n"
        return Comprobantecompra

In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
